<a href="https://colab.research.google.com/github/Ihoold/pixelgen/blob/master/funit4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from unittest import mock
import os
import numpy as np
import tensorflow as tf
import json
import matplotlib.pyplot as plt
from tensorflow.python.estimator import estimator
from google.colab import drive, auth

In [0]:
R, C = 4, 3
EXAMPLES = R * C
    
CHANNELS = 4
LATENT_DIM = 128
ADD_NOISE_TO_EXAMPLE = False

DROPOUT_PROB = 0.5
ALPHA = 0.2
BATCH_SIZE = 1024
EPOCHS = 100000
EVAL_EPOCHS = 1000
G_LR = 0.0002
D_LR = 0.0001
KERNEL_SIZE = 4
NUM_CLASSES = 21

MODEL_NAME = 'FUNIT'
RUN_NAME = 'FUNIT_only_gen_10'

data_train_file = 'gs://tputestingmnist/datasets/dataset_train_funit.tfrecords'
data_test_file = 'gs://tputestingmnist/datasets/dataset_test_funit.tfrecords'
MODEL_DIR = 'gs://tputestingmnist/{}/{}/'.format(MODEL_NAME, RUN_NAME)
# MODEL_DIR = 'gs://tputestingmnist/{}/'.format(RUN_NAME)
GOOGLE_DRIVE_DIR = '/content/gdrive/My Drive/Programowanie/PixelGen/{}'.format(RUN_NAME)
TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])

In [0]:
#################################### SETUP #####################################

def setup():
    drive.mount('/content/gdrive')
    auth.authenticate_user()


def upload_credentials():
    # Upload credentials to TPU.
    with tf.Session(TF_MASTER) as sess:    
        with open('/content/adc.json', 'r') as f:
            auth_info = json.load(f)
        tf.contrib.cloud.configure_gcs(sess, credentials=auth_info)

In [0]:
################################# DATA INPUT ###################################

def parser(serialized_example):
        """Parses a single example into image and label tensors."""
        features = tf.parse_single_example(
            serialized_example,
            features={
                'image_content': tf.FixedLenFeature([], tf.string),
                'image_style': tf.FixedLenFeature([], tf.string),
                'label': tf.FixedLenFeature([], tf.int64) 
            })

        content_image = tf.decode_raw(features['image_content'], tf.uint8)
        content_image.set_shape([48 * 48 * 4])
        content_image = tf.reshape(content_image, [48, 48, 4])[:,:,:CHANNELS]
        # Normalize the values of the image from [0, 255] to [-1.0, 1.0]
        content_image = tf.cast(content_image, dtype=tf.float32) / 127.5 - 1

        style_image = tf.decode_raw(features['image_style'], tf.uint8)
        style_image.set_shape([48 * 48 * 4])
        style_image = tf.reshape(style_image, [48, 48, 4])[:,:,:CHANNELS]
        # Normalize the values of the image from [0, 255] to [-1.0, 1.0]
        style_image = tf.cast(style_image, dtype=tf.float32) / 127.5 - 1
        
        label = features['label']

        return {'style_images': style_image,
                'content_images': content_image}, label


def make_input_fn(is_training=True, one_batch=False):
    def input_fn(params):
        batch_size = params['batch_size']
        dataset = tf.data.TFRecordDataset(data_train_file, buffer_size=8*1024*1024)
        dataset = dataset.map(parser).cache().shuffle(batch_size)
        if is_training:
            dataset = dataset.repeat()
        if one_batch:
            dataset = dataset.take(params['batch_size'])
            
        dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(8)

        return dataset
    return input_fn


def predict_input_fn(params):
    batch_size = params['batch_size']
    dataset = tf.data.TFRecordDataset(data_test_file).map(parser).cache().shuffle(batch_size)
    style_images, content_images, _ = dataset.prefetch(batch_size).batch(batch_size, drop_remainder=True).make_one_shot_iterator().get_next()
    
    features = {
        'style_images': style_images,
        'content_images': content_images,
    }
    return features, None


In [0]:
############################### DATA SAVEING ###################################
 
def images_to_zero_one(images):
        return np.clip(np.array(images) * 0.5 + 0.5, 0., 1.)


def save_imgs(epoch, images):
    if not os.path.exists(GOOGLE_DRIVE_DIR):
        os.mkdir(GOOGLE_DRIVE_DIR)

    # Rescale images to 0 - 1
    images = images_to_zero_one(images)
    fig, axs = plt.subplots(R, C, figsize=(20,20))

    for i in range(R):
        for j in range(C):
            axs[i,j].imshow(images[C*i + j])
            axs[i,j].axis('off')

    fig.savefig(os.path.join(GOOGLE_DRIVE_DIR, '{}.png'.format(epoch)))
    plt.close()

In [0]:
################################## MODEL #######################################

def _relu(x):
    return tf.nn.relu(x)


def _leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=ALPHA)


def _batch_norm(x, is_training, name):
    return tf.layers.batch_normalization(x, momentum=0.8, epsilon=1e-5, 
                                         training=is_training, name=name)

def _make_batch_norm(is_training, name):
    def bn(x):
        return _batch_norm(x, is_training=is_training, name=name)
    return bn

def _instance_norm(x):
    return tf.contrib.layers.instance_norm(x)


def _make_adain_norm(style_mean, style_var):
    def _adain_norm(x):
        mean, var = tf.nn.moments(x, axes=[0])
        return tf.nn.batch_normalization(x, mean, var, style_mean, style_var)
    return _adain_norm


def _dense(x, neurons, name, activation=None):
    return tf.layers.dense(x, neurons, name=name, activation=activation,
                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))


def _conv2d(x, filters, kernel_size, stride, name, activation=None):
    return tf.layers.conv2d(x, filters, [kernel_size, kernel_size], 
                            strides=[stride, stride], activation=activation,
                            padding='same', name=name,
                            kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))

def _pooling(x, kernel, stride, index):
    return tf.layers.average_pooling2d(x, pool_size=kernel, strides=stride, name='pool_{}'.format(index))

def _deconv2d(x, filters, stride, name, activation=None):
    return tf.layers.conv2d_transpose(x, filters, [KERNEL_SIZE, KERNEL_SIZE],
                                      strides=[stride, stride], activation=activation,
                                      padding='same', name=name,
                                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))


def convolution_block(x, filters, kernel_size, resize_factor, index, activation=_relu, activation_first=False, normalization=None):
    if activation and activation_first:
        x = activation(x)
    x = _conv2d(x, kernel_size=kernel_size, filters=filters, stride=resize_factor, activation=None, name='conv_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation and not activation_first:
        x = activation(x)
    return x


def deconvolution_block(x, filters, resize_factor, index, activation=_relu, normalization=None):
    x = _deconv2d(x, filters=filters, stride=resize_factor, activation=None, name='deconv_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation:
        x = activation(x)
    return x


def fully_connected_block(x, neurons, index, activation=_relu, normalization=None):
    x = _dense(x, neurons=neurons, activation=None, name='fc_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation:
        x = activation(x)
    return x


def residual_block(x, filters_in, filters_out, index, kernel_size=3, activation=_relu, activation_first=False, normalization=None):
    forwarded_x = convolution_block(x, filters_in, kernel_size, resize_factor=1, 
                                    index='res_con1_{}'.format(index), 
                                    activation_first=activation_first, activation=activation)
    
    forwarded_x = convolution_block(forwarded_x, filters_out, kernel_size, resize_factor=1, 
                                    index='res_con2_{}'.format(index), 
                                    activation_first=activation_first, activation=None)
    
    if filters_in != filters_out:
        x = convolution_block(x, filters_out, kernel_size=1, resize_factor=1, index='res_con3_{}'.format(index), activation=None)
    x = tf.add(forwarded_x, x)
    return x


def adain_residual_block(x, mean, var, filters_in, filters_out, index, activation=_relu, activation_first=False):
    with tf.variable_scope('adain_{}'.format(index)):
        normalization = _make_adain_norm(mean, var)
        return residual_block(x, filters_in, filters_out, index, 3, activation, activation_first, normalization)


def content_encoder(x):
    x = convolution_block(x, 64, kernel_size=7, resize_factor=2, normalization=_instance_norm, index='con1')
    x = convolution_block(x, 128, kernel_size=4, resize_factor=2, normalization=_instance_norm, index='con2')
    x = convolution_block(x, 256, kernel_size=4, resize_factor=2, normalization=_instance_norm, index='con3')
    x = convolution_block(x, 512, kernel_size=4, resize_factor=2, normalization=_instance_norm, index='con4')
    
    x = residual_block(x, 512, 512, index='res1', normalization=_instance_norm)
    x = residual_block(x, 512, 512, index='res2', normalization=_instance_norm)
    return x


def class_encoder(x):
    x = convolution_block(x, 64, kernel_size=7, resize_factor=2, index='cls1')
    x = convolution_block(x, 128, kernel_size=4, resize_factor=2, index='cls2')
    x = convolution_block(x, 256, kernel_size=4, resize_factor=2, index='cls3')
    x = convolution_block(x, 512, kernel_size=4, resize_factor=2, index='cls4')
    x = convolution_block(x, 1024, kernel_size=4, resize_factor=2, index='cls5')
    x = _pooling(x, 3, 3, index='cls6')
    return x


def decoder(content_latent):    
    class_latent = fully_connected_block(class_latent, 256, 'dec11')
    class_latent = fully_connected_block(class_latent, 256, 'dec12')
    class_latent = fully_connected_block(class_latent, 256, 'dec13')
    mean, var = tf.nn.moments(class_latent, axes=[0])
    
    x = adain_residual_block(content_latent, mean, var, 512, 512, index='res3', activation=None)
    x = adain_residual_block(x, mean, var, 512, 512, index='res4', activation=None)

    x = deconvolution_block(content_latent, 512, resize_factor=1, normalization=_instance_norm, index='dec0')
    x = deconvolution_block(x, 256, resize_factor=2, normalization=_instance_norm, index='dec1')
    x = deconvolution_block(x, 128, resize_factor=2, normalization=_instance_norm, index='dec2')
    x = deconvolution_block(x, 64, resize_factor=2, normalization=_instance_norm, index='dec3')
    x = deconvolution_block(x, CHANNELS, resize_factor=1, normalization=_instance_norm, index='dec4', activation=tf.tanh)
    
    return x


class Funit:

    @staticmethod
    def discriminator(x, label, scope='Discriminator'):
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            x = convolution_block(x, 64, kernel_size=7, resize_factor=1, index='disc_11')
            
            x = residual_block(x, 64, 64, 'disc_12', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 64, 128, 'disc_13', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_14')
            
            x = residual_block(x, 128, 128, 'disc_15', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 128, 256, 'disc_16', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_17')
            
            x = residual_block(x, 256, 256, 'disc_18', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 256, 512, 'disc_19', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_20')
            
            x = residual_block(x, 512, 512, 'disc_21', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 512, 1024, 'disc_22', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_23')
            
            x = residual_block(x, 1024, 1024, 'disc_24', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 1024, 1024, 'disc_25', kernel_size=3, activation=_leaky_relu, activation_first=True)
            
            x = _leaky_relu(x)
            x = tf.layers.flatten(x)
            x = _dense(x, NUM_CLASSES, 'disc_out')
            label = tf.expand_dims(label, axis=1)
            x = tf.gather_nd(x, label, batch_dims=1)
            
            return x
          
    @staticmethod
    def generator(image_content, image_style, is_training, scope='Generator'):
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
#             class_latent = class_encoder(image_style)
            
#             x = decoder(content_latent)

            x = content_encoder(image_style)
        
            x = deconvolution_block(x, 512, resize_factor=2, index='deconv21', normalization=_instance_norm)
            x = deconvolution_block(x, 256, resize_factor=2, index='deconv22', normalization=_instance_norm)
            x = deconvolution_block(x, 128, resize_factor=2, index='deconv23', normalization=_instance_norm)
            x = deconvolution_block(x, CHANNELS, resize_factor=2, index='deconv24', activation=tf.tanh, normalization=None)
            return x

In [0]:
################################ MODEL FUN #####################################
def make_model_fn(model):

    def model_fn(features, labels, mode, params):
        with tf.variable_scope('inputs'):
            content_images = features['content_images']
            style_images = features['style_images']
        generated_images = model.generator(content_images, style_images, (mode == tf.estimator.ModeKeys.TRAIN))
        
        # PREDICT #
        if mode == tf.estimator.ModeKeys.PREDICT:
            predictions = {
                'generated_images': generated_images,
                'content_images': content_images,
                'style_images': style_images
            }
            return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, predictions=predictions)

        # Discriminator loss
        d_on_data_logits = tf.squeeze(model.discriminator(style_images, labels))
        d_on_g_logits = tf.squeeze(model.discriminator(generated_images, labels))

        
        with tf.variable_scope('losses'):
            d_loss = tf.contrib.gan.losses.wargs.modified_discriminator_loss(
                discriminator_real_outputs=d_on_data_logits,
                discriminator_gen_outputs=d_on_g_logits,
                reduction=tf.losses.Reduction.NONE,
                label_smoothing=0.2
            )
            d_loss_reduced = tf.reduce_mean(d_loss)

            # Generator loss
            g_loss = tf.contrib.gan.losses.wargs.modified_generator_loss(
                discriminator_gen_outputs=d_on_g_logits,
                reduction=tf.losses.Reduction.NONE
            )
            g_loss_reduced = tf.reduce_mean(g_loss)
            
        # TRAIN #
        if mode == tf.estimator.ModeKeys.TRAIN:
            
            with tf.variable_scope('optimizer'):
                d_optimizer = tf.train.AdamOptimizer(learning_rate=D_LR, beta1=0.5)
                d_optimizer = tf.contrib.tpu.CrossShardOptimizer(d_optimizer)
            
                g_optimizer = tf.train.AdamOptimizer(learning_rate=G_LR, beta1=0.5)
                g_optimizer = tf.contrib.tpu.CrossShardOptimizer(g_optimizer)
         
                with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
                    d_step = d_optimizer.minimize(d_loss_reduced, var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                                                                                     scope='Discriminator'))
                    g_step = g_optimizer.minimize(g_loss_reduced, var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                                                                                     scope='Generator'))
                    increment_step = tf.assign_add(tf.train.get_or_create_global_step(), 1)
                    joint_op = tf.group([d_step, g_step, increment_step])

                    a = tf.contrib.tpu.TPUEstimatorSpec(mode=mode, loss=d_loss_reduced+g_loss_reduced, train_op=joint_op)
                    return a

        # EVAL #
        elif mode == tf.estimator.ModeKeys.EVAL:
            def _eval_metric_fn(d_loss, g_loss, d_real_labels, d_gen_lanels, d_real_logits, d_gen_logits):
                return {
                    'discriminator_loss': tf.metrics.mean(d_loss),
                    'generator_loss': tf.metrics.mean(g_loss),
                    'discriminator_real_accuracy': tf.metrics.accuracy(labels=d_real_labels, predictions=tf.math.round(tf.sigmoid(d_real_logits))),
                    'discriminator_gen_accuracy': tf.metrics.accuracy(labels=d_gen_lanels, predictions=tf.math.round(tf.sigmoid(d_gen_logits))),
                }

            return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, loss=d_loss_reduced + g_loss_reduced,
                                                   eval_metrics=(_eval_metric_fn, [d_loss, g_loss, 
                                                                                   tf.ones_like(labels), tf.zeros_like(labels),
                                                                                   d_on_data_logits, d_on_g_logits]))
    return model_fn

In [0]:
################################ ESTIMATORS ####################################

def make_estimators(model, only_cpu=False):
    model_fn = make_model_fn(model)
    
    config = tf.contrib.tpu.RunConfig(
        master=TF_MASTER,
        save_checkpoints_steps=EVAL_EPOCHS,
        save_checkpoints_secs=None,
        save_summary_steps=EVAL_EPOCHS,
        model_dir=MODEL_DIR,
        keep_checkpoint_max=3,
        tpu_config=tf.contrib.tpu.TPUConfig(iterations_per_loop=EVAL_EPOCHS))

    if not only_cpu:
        # TPU-based estimator used for TRAIN and EVAL
        est = tf.contrib.tpu.TPUEstimator(
            model_fn=model_fn,
            use_tpu=True,
            config=config,
            train_batch_size=BATCH_SIZE,
            eval_batch_size=BATCH_SIZE)
    else:
        est = None

    # CPU-based estimator used for PREDICT (generating images)
    cpu_est = tf.contrib.tpu.TPUEstimator(
        model_fn=model_fn,
        use_tpu=False,
        config=config,
        predict_batch_size=EXAMPLES)
    
    return est, cpu_est

In [0]:
################################# TRAINING #####################################

def train(est, cpu_est):
    current_step = estimator._load_global_step_from_checkpoint_dir(MODEL_DIR)
    print('Starting training')

    while current_step < EPOCHS:
        next_checkpoint = int(min(current_step + EVAL_EPOCHS, EPOCHS))
        est.train(input_fn=make_input_fn(), max_steps=next_checkpoint)
        current_step = next_checkpoint
        print('Finished training step %d' % current_step)

        # Evaluation
        metrics = est.evaluate(input_fn=make_input_fn(False), steps=1)
        print('Finished evaluating')
        print(metrics)

        # Render some generated images
        generated_iter = cpu_est.predict(input_fn=make_input_fn(False, one_batch=True))
        images = [np.concatenate([p['content_images'], p['style_images'], p['generated_images']], axis=1) for p in generated_iter]
        save_imgs(str(current_step), images)
        print('Finished generating images')

In [0]:
def do_experiment():
    setup()
    upload_credentials()
    model = Funit()
    est, cpu_est = make_estimators(model)
    train(est, cpu_est)

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)

try:
    do_experiment()
except Exception as e:
    print (e)
    pass

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


W0818 13:32:53.219227 140213043791744 estimator.py:1984] Estimator's model_fn (<function make_model_fn.<locals>.model_fn at 0x7f85ab5e3bf8>) includes params argument, but params are not passed to Estimator.
I0818 13:32:53.221008 140213043791744 estimator.py:209] Using config: {'_model_dir': 'gs://tputestingmnist/FUNIT/FUNIT_only_gen_10/', '_tf_random_seed': None, '_save_summary_steps': 1000, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8596855668>, '_task_type': 'worker', '_task_i

Starting training


I0818 13:32:53.592084 140213043791744 estimator.py:1145] Calling model_fn.
I0818 13:33:01.607141 140213043791744 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0818 13:33:01.794985 140213043791744 estimator.py:1147] Done calling model_fn.
I0818 13:33:03.828596 140213043791744 tpu_estimator.py:499] TPU job name tpu_worker
I0818 13:33:04.770968 140213043791744 monitored_session.py:240] Graph was finalized.
I0818 13:33:07.308872 140213043791744 session_manager.py:500] Running local_init_op.
I0818 13:33:07.619271 140213043791744 session_manager.py:502] Done running local_init_op.
I0818 13:33:11.771410 140213043791744 basic_session_run_hooks.py:606] Saving checkpoints for 0 into gs://tputestingmnist/FUNIT/FUNIT_only_gen_10/model.ckpt.
W0818 13:33:24.083506 140213043791744 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/tpu/tpu_estimator.py:741: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be r

Finished training step 1000


...

I0818 13:50:03.140367 140213043791744 estimator.py:1147] Done calling model_fn.
I0818 13:50:03.168457 140213043791744 evaluation.py:255] Starting evaluation at 2019-08-18T13:50:03Z
I0818 13:50:03.169845 140213043791744 tpu_estimator.py:499] TPU job name tpu_worker
I0818 13:50:03.463433 140213043791744 monitored_session.py:240] Graph was finalized.
I0818 13:50:03.525415 140213043791744 saver.py:1280] Restoring parameters from gs://tputestingmnist/FUNIT/FUNIT_only_gen_10/model.ckpt-3000
I0818 13:50:07.320189 140213043791744 session_manager.py:500] Running local_init_op.
I0818 13:50:07.424774 140213043791744 session_manager.py:502] Done running local_init_op.
I0818 13:50:07.616592 140213043791744 tpu_estimator.py:557] Init TPU system
I0818 13:50:15.333358 140213043791744 tpu_estimator.py:566] Initialized TPU in 7 seconds
I0818 13:50:15.335003 140211672889088 tpu_estimator.py:514] Starting infeed thread controller.
I0818 13:50:15.335632 140211664496384 tpu_estimator.py:533] Starting outfee

Finished evaluating
{'discriminator_gen_accuracy': 1.0, 'discriminator_loss': 1.0676461, 'discriminator_real_accuracy': 0.5986328, 'generator_loss': 0.9613806, 'loss': 2.012992, 'global_step': 3000}


I0818 13:50:26.009825 140213043791744 estimator.py:1147] Done calling model_fn.
I0818 13:50:26.165622 140213043791744 monitored_session.py:240] Graph was finalized.
I0818 13:50:26.220114 140213043791744 saver.py:1280] Restoring parameters from gs://tputestingmnist/FUNIT/FUNIT_only_gen_10/model.ckpt-3000
I0818 13:50:26.512659 140213043791744 session_manager.py:500] Running local_init_op.
I0818 13:50:26.526628 140213043791744 session_manager.py:502] Done running local_init_op.
I0818 13:50:26.815415 140213043791744 error_handling.py:96] prediction_loop marked as finished
I0818 13:50:26.816714 140213043791744 error_handling.py:96] prediction_loop marked as finished


Finished generating images


I0818 13:50:28.039773 140213043791744 estimator.py:1145] Calling model_fn.
I0818 13:50:35.930735 140213043791744 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0818 13:50:36.116376 140213043791744 estimator.py:1147] Done calling model_fn.
I0818 13:50:36.117794 140213043791744 tpu_estimator.py:499] TPU job name tpu_worker
I0818 13:50:37.089671 140213043791744 monitored_session.py:240] Graph was finalized.
I0818 13:50:37.271565 140213043791744 saver.py:1280] Restoring parameters from gs://tputestingmnist/FUNIT/FUNIT_only_gen_10/model.ckpt-3000
I0818 13:50:42.402595 140213043791744 session_manager.py:500] Running local_init_op.
I0818 13:50:42.758578 140213043791744 session_manager.py:502] Done running local_init_op.
I0818 13:50:47.214097 140213043791744 basic_session_run_hooks.py:606] Saving checkpoints for 3000 into gs://tputestingmnist/FUNIT/FUNIT_only_gen_10/model.ckpt.
I0818 13:51:04.309714 140213043791744 util.py:98] Initialized dataset iterators in 0 seconds
I0818 13:5